In [ ]:
# !pip install bayesian-optimization==1.1.0  -t $HOME/external-libraries/bayesian-optimization
# !pip install toad==0.0.64  -t $HOME/external-libraries/toad
# !pip install catboost==0.26.1 -t $HOME/external-libraries/catboost
# !pip install ipywidgets==7.6.5 -t $HOME/external-libraries/ipywidgets
# !pip install pandas-profiling==3.1.0 -t $HOME/external-libraries/pandas-profiling

In [1]:
import sys
sys.path.append('/home/aistudio/external-libraries/catboost103')
sys.path.append('/home/aistudio/external-libraries/ipywidgets')
sys.path.append('/home/aistudio/external-libraries/bayesian-optimization')
sys.path.append('/home/aistudio/external-libraries/toad')
sys.path.append('/home/aistudio/external-libraries/pandas-profiling')

In [2]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib # 注意这个也要import一次
import matplotlib.pyplot as plt 
import seaborn as sns
import pickle
from sklearn import metrics
from sklearn.metrics import roc_auc_score, accuracy_score, recall_score, mean_squared_error, roc_curve, auc
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import lightgbm as lgb
import toad
from toad.plot import bin_plot, badrate_plot, proportion_plot
import pandas_profiling as pp
from bayes_opt import BayesianOptimization
# from skopt.space import Real, Categorical, Integer
import os
import shutil
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)

In [ ]:
# reduce_mem_usage 函数通过调整数据类型，帮助我们减少数据在内存中占用的空间
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() 
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
#         else:
#             df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() 
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
# 导入数据
train_data = pd.read_csv('./data/data119778/train.csv', index_col= 'id')
testA_data = pd.read_csv('./data/data119778/testA.csv', index_col= 'id')

In [ ]:
train_data = reduce_mem_usage(train_data)
testA_data = reduce_mem_usage(testA_data)

Memory usage of dataframe is 300800000.00 MB
Memory usage after optimization is: 102400000.00 MB
Decreased by 66.0%
Memory usage of dataframe is 73600000.00 MB
Memory usage after optimization is: 25400000.00 MB
Decreased by 65.5%


In [ ]:
toad.detect(train_data)

,type,size,missing,unique,mean_or_top1,std_or_top2,min_or_top3,1%_or_top4,10%_or_top5,50%_or_bottom5,75%_or_bottom4,90%_or_bottom3,99%_or_bottom2,max_or_bottom1
loanAmnt,float16,800000,0.00%,1488,NaN,NaN,500,1500,5000,12000,20000,28000,35008,40000
term,int8,800000,0.00%,2,3.48274,0.855832,3,3,3,3,3,5,5,5
interestRate,float16,800000,0.00%,620,NaN,0,5.30859,5.32031,7.39062,12.7422,15.9922,19.4844,26.2969,30.9844
installment,float16,800000,0.00%,5589,NaN,NaN,15.6875,53.1562,157.625,375.25,580.5,807.5,1220,1715
grade,object,800000,0.00%,7,B:29.21%,C:28.39%,A:17.46%,D:14.93%,E:6.96%,A:17.46%,D:14.93%,E:6.96%,F:2.38%,G:0.67%
subGrade,object,800000,0.00%,35,C1:6.35%,B4:6.19%,B5:6.12%,B3:6.08%,C2:5.88%,G1:0.22%,G2:0.15%,G3:0.12%,G4:0.09%,G5:0.08%
employmentTitle,float32,800000,0.00%,248683,72005.4,106586,0,19,54,7755,117664,256721,365010,378351
employmentLength,object,800000,5.85%,11,10+ years:32.84%,2 years:9.04%,< 1 year:8.03%,3 years:8.02%,1 year:6.56%,4 years:6.00%,6 years:4.66%,8 years:4.52%,7 years:4.43%,9 years:3.78%
homeOwnership,int8,800000,0.00%,6,0.614213,0.675749,0,0,0,1,1,2,2,5
annualIncome,float32,800000,0.00%,44926,76133.9,68947.5,0,18000,34000,65000,90000,125000,250000,1.09992e+07


In [ ]:
# iv_info = toad.quality(train_data,'isDefault', iv_only=True)
# iv_info.sort_values(by='iv', ascending=False)

,iv,gini,entropy,unique
subGrade,0.499203,NaN,NaN,35.0
grade,0.463477,NaN,NaN,7.0
interestRate,0.462692,NaN,NaN,620.0
term,0.172635,NaN,NaN,2.0
ficoRangeHigh,0.125253,NaN,NaN,39.0
ficoRangeLow,0.125253,NaN,NaN,39.0
dti,0.072863,NaN,NaN,4179.0
verificationStatus,0.054519,NaN,NaN,3.0
issueDate,0.053335,NaN,NaN,139.0
n14,0.050250,NaN,NaN,32.0


In [ ]:
train = train_data.copy()
testA = testA_data.copy()

## 一、特征工程

### 1.特征标签编码

In [ ]:
#标签编码 grade、subGrade
grade_map = dict(zip(('A', 'B', 'C', 'D', 'E', 'F', 'G'), (0, 5, 10, 15, 20, 25, 30, 35)))
subGrade_map =dict(zip(('A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C1', 'C2', 'C3', 'C4', 'C5', 'D1', 'D2', 'D3', 'D4', 'D5', 'E1', 'E2','E3', 'E4', 'E5', 'F1', 'F2', 'F3', 'F4', 'F5', 'G1', 'G2', 'G3', 'G4', 'G5'),
(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34)))

In [ ]:
train['grade'] = train['grade'].map(grade_map)
testA['grade'] = testA['grade'].map(grade_map)

train['subGrade'] = train['subGrade'].map(subGrade_map)
testA['subGrade'] = testA['subGrade'].map(subGrade_map)

In [ ]:
# 标签编码 employmentLength
employmentLength_map = dict(zip(( '< 1 year', '1 year', '2 years', '3 years', '4 years', '5 years', '6 years', '7 years', '8 years', '9 years', '10+ years'),
(0 , 1, 2, 3, 4, 5, 6, 7, 8, 9, 10)))

In [ ]:
train['employmentLength'] = train['employmentLength'].map(employmentLength_map)
testA['employmentLength'] = testA['employmentLength'].map(employmentLength_map)

In [ ]:
#标签编码 issueDate	
train['issueDate'] = pd.to_datetime(train['issueDate'])
testA['issueDate'] = pd.to_datetime(testA['issueDate'])

In [ ]:
train['issueDate_year'] = pd.to_datetime(train['issueDate']).dt.year
train['issueDate_month'] = pd.to_datetime(train['issueDate']).dt.month

testA['issueDate_year'] = pd.to_datetime(testA['issueDate']).dt.year
testA['issueDate_month'] = pd.to_datetime(testA['issueDate']).dt.month

In [ ]:
base_time = pd.datetime.strptime('2007-06-01', '%Y-%m-%d')
train['issueDate_diff'] = train['issueDate'].apply(lambda x: (x - base_time).days)
testA['issueDate_diff'] = testA['issueDate'].apply(lambda x: (x - base_time).days)

In [ ]:
# 标签编码 earliesCreditLine
train['earliesCreditLine'] = pd.to_datetime(train['earliesCreditLine'])
testA['earliesCreditLine'] = pd.to_datetime(testA['earliesCreditLine'])

In [ ]:
train['earliesCreditLine_year'] = pd.to_datetime(train['earliesCreditLine']).dt.year
train['earliesCreditLine_month'] = pd.to_datetime(train['earliesCreditLine']).dt.month

testA['earliesCreditLine_year'] = pd.to_datetime(testA['earliesCreditLine']).dt.year
testA['earliesCreditLine_month'] = pd.to_datetime(testA['earliesCreditLine']).dt.month

In [ ]:
base_time2 = pd.datetime.strptime('1944-01-01', '%Y-%m-%d')
train['earliesCreditLine_diff'] = train['earliesCreditLine'].apply(lambda x: (x - base_time2).days)
testA['earliesCreditLine_diff'] = testA['earliesCreditLine'].apply(lambda x: (x - base_time2).days)

In [ ]:
# 使用FreqEncode 编码employmentTitle、title
f_en_list = ['employmentTitle', 'title']
for col in f_en_list:
    merge_data = pd.concat((train[[col]], testA[[col]]))
    col_map = merge_data[col].value_counts(dropna=False,normalize=True).to_dict()
#     temp1 = data[col].value_counts().to_dict()
    train['{}_freq_encode'.format(col)] = train[col].map(col_map)
    testA['{}_freq_encode'.format(col)] = testA[col].map(col_map)

### 2.业务特征

In [ ]:
train['earliesCreditLine_issueDate_diff'] = (train.issueDate - train.earliesCreditLine).apply(lambda x:x.days)
testA['earliesCreditLine_issueDate_diff'] = (testA.issueDate - testA.earliesCreditLine).apply(lambda x:x.days)

In [ ]:
train['loan_2_income'] = train.loanAmnt / (train.annualIncome)
testA['loan_2_income'] = testA.loanAmnt / (testA.annualIncome)

In [ ]:
train['loan_2_income_per_year'] = train.loanAmnt / train.term / (train.annualIncome)
testA['loan_2_income_per_year'] = testA.loanAmnt / testA.term / (testA.annualIncome)

In [ ]:
train['installment_2_income'] = train.installment * 12 / (train.annualIncome)
testA['installment_2_income'] = testA.installment * 12 / (testA.annualIncome)

In [ ]:
train['openAcc_totalAcc_rate'] = train.openAcc / train.totalAcc
testA['openAcc_totalAcc_rate'] = testA.openAcc / testA.totalAcc

In [ ]:
train['pubRec_diff'] = train.pubRec - train.pubRecBankruptcies
testA['pubRec_diff'] = testA.pubRec - testA.pubRecBankruptcies

In [ ]:
train['revolamnt'] = train.revolBal / train.revolUtil * 100
testA['revolamnt'] = testA.revolBal / testA.revolUtil * 100

In [ ]:
train['revol_2_income'] = train.revolBal / (train.annualIncome)
testA['revol_2_income'] = testA.revolBal / (testA.annualIncome)

In [ ]:
train['installment_revol_2_income'] = (train.installment + train.revolBal) / ((train.annualIncome) / 12)
testA['installment_revol_2_income'] = (testA.installment + train.revolBal) / ((testA.annualIncome) / 12)

In [ ]:
train['total_income'] = train.annualIncome * train.employmentLength
testA['total_income'] = testA.annualIncome * testA.employmentLength

In [ ]:
train['null_num'] = train.isnull().sum(axis=1)
testA['null_num'] = testA.isnull().sum(axis=1)

In [ ]:
train['end_year'] = train.issueDate_year + train.term
testA['end_year'] = testA.issueDate_year + testA.term

In [ ]:
train.drop(columns='issueDate', inplace = True)
testA.drop(columns='issueDate', inplace = True)
train.drop(columns='earliesCreditLine', inplace = True)
testA.drop(columns='earliesCreditLine', inplace = True)

In [ ]:
def inf_2_nan(df):
    for col, inf_cnt in (np.isinf(df.select_dtypes(exclude='O')).sum()).to_dict().items():
        if inf_cnt >0:
            df[col][np.isinf(df[col])] = np.nan

In [ ]:
inf_2_nan(train)
inf_2_nan(testA)

### 3.保存初步特征工程后的数据

In [ ]:
X_train = train.drop(columns=['isDefault'])
y_train = train[['isDefault']]

X_testA = testA.copy()

In [ ]:
with open('./work/features/X_train.csv', 'wb') as file:
    pickle.dump(X_train, file)
    
with open('./work/features/y_train.csv', 'wb') as file:
    pickle.dump(y_train, file)
    
with open('./work/features/X_testA.csv', 'wb') as file:
    pickle.dump(X_testA, file)

### 4. 分箱(耗时太长，暂未完成)

#### 4.1 分箱

In [ ]:
X_train_s, X_eval,y_train_s, y_eval = train_test_split(X_train,y_train,test_size=0.2,random_state=1, stratify=y_train)

In [ ]:
data_tr = pd.concat((X_train_s, y_train_s), axis=1)
data_tr['type'] = 'train'

data_ts = pd.concat((X_eval,y_eval), axis=1)
data_ts['type'] = 'eval'

In [ ]:
# feat_lst = list(X_eval.columns)
# psi_df = toad.metrics.PSI(X_train_s[feat_lst], X_eval[feat_lst]).sort_values(0)

In [ ]:
# psi_df[psi_df.values > 0.3]

In [ ]:
bins = dict()

# 初始化一个combiner类
combiner = toad.transform.Combiner()

In [ ]:
# 训练数据并指定分箱方法，其它参数可选
# combiner.fit(data_tr, y='isDefault', method='chi', min_samples =  0.05, exclude = 'type')

for col in list(data_tr.columns):
    print('col:', col)
    combiner.fit(data_tr[[col, 'isDefault']], y='isDefault', method='chi', min_samples =  0.05)
    b = combiner.export()
    print(b)
    bins.update(combiner.export())

col: loanAmnt
{'loanAmnt': [3524.0, 9504.0, 10024.0, 15024.0, 28032.0]}
col: term
{'term': [5]}
col: interestRate
{'interestRate': [7.890625, 10.40625, 12.59375, 14.4609375, 17.796875]}
col: installment
{'installment': [162.0, 251.5, 301.25, 335.0, 501.0, 602.5, 661.5]}
col: grade
{'grade': [5, 10, 15, 20]}
col: subGrade
{'subGrade': [3, 5, 8, 10, 12, 15, 18, 21]}
col: employmentTitle
{'employmentTitle': [55.0, 211519.0]}
col: employmentLength
{'employmentLength': [0.0, 2.0, 5.0, 7.0, 10.0]}
col: homeOwnership
{'homeOwnership': [1, 2]}
col: annualIncome
{'annualIncome': [36380.0, 45302.0, 60005.0, 65727.0, 85000.078125, 100079.0, 125660.0]}
col: verificationStatus
{'verificationStatus': [1, 2]}
col: purpose
{'purpose': [2]}
col: postCode
{'postCode': [21.0, 130.0, 272.0]}
col: regionCode
{'regionCode': [11, 22]}
col: dti
{'dti': [11.8125, 19.234375, 24.5]}
col: delinquency_2years
{'delinquency_2years': [1.0, 2.0]}
col: ficoRangeLow
{'ficoRangeLow': [670.0, 685.0, 695.0, 710.0, 725.0, 7

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [ ]:
bins

{'loanAmnt': [3524.0, 9504.0, 10024.0, 15024.0, 28032.0],
 'term': [5],
 'interestRate': [7.890625, 10.40625, 12.59375, 14.4609375, 17.796875],
 'installment': [162.0, 251.5, 301.25, 335.0, 501.0, 602.5, 661.5],
 'grade': [5, 10, 15, 20],
 'subGrade': [3, 5, 8, 10, 12, 15, 18, 21],
 'employmentTitle': [55.0, 211519.0],
 'employmentLength': [0.0, 2.0, 5.0, 7.0, 10.0],
 'homeOwnership': [1, 2],
 'annualIncome': [36380.0,
  45302.0,
  60005.0,
  65727.0,
  85000.078125,
  100079.0,
  125660.0],
 'verificationStatus': [1, 2],
 'purpose': [2],
 'postCode': [21.0, 130.0, 272.0],
 'regionCode': [11, 22],
 'dti': [11.8125, 19.234375, 24.5],
 'delinquency_2years': [1.0, 2.0],
 'ficoRangeLow': [670.0, 685.0, 695.0, 710.0, 725.0, 740.0],
 'ficoRangeHigh': [674.0, 689.0, 699.0, 714.0, 729.0, 744.0],
 'openAcc': [11.0],
 'pubRec': [1.0],
 'pubRecBankruptcies': [1.0],
 'revolBal': [23875.0],
 'revolUtil': [19.796875, 29.59375, 39.59375, 51.90625, 76.0],
 'totalAcc': [17.0, 22.0],
 'initialListStatus

#### 4.1.1保存、读取bins信息

In [ ]:
# with open('./work/features/bins.csv', 'wb') as file:
#     pickle.dump(bins, file)

In [ ]:
with open('./work/features/bins.csv', 'rb') as file:
    bins = pickle.load(file)

#### 4.2 分箱可视化化，调整分箱

In [ ]:
c2 = toad.transform.Combiner()
c2.set_rules(bins)

data_ = pd.concat((data_tr, data_ts))
temp_data = c2.transform(data_)

In [ ]:
temp_data.columns

Index(['loanAmnt', 'term', 'interestRate', 'installment', 'grade', 'subGrade',
       'employmentTitle', 'employmentLength', 'homeOwnership', 'annualIncome',
       'verificationStatus', 'purpose', 'postCode', 'regionCode', 'dti',
       'delinquency_2years', 'ficoRangeLow', 'ficoRangeHigh', 'openAcc',
       'pubRec', 'pubRecBankruptcies', 'revolBal', 'revolUtil', 'totalAcc',
       'initialListStatus', 'applicationType', 'title', 'policyCode', 'n0',
       'n1', 'n2', 'n3', 'n4', 'n5', 'n6', 'n7', 'n8', 'n9', 'n10', 'n11',
       'n12', 'n13', 'n14', 'issueDate_year', 'issueDate_month',
       'issueDate_diff', 'earliesCreditLine_year', 'earliesCreditLine_month',
       'earliesCreditLine_diff', 'employmentTitle_freq_encode',
       'title_freq_encode', 'earliesCreditLine_issueDate_diff',
       'loan_2_income', 'loan_2_income_per_year', 'installment_2_income',
       'openAcc_totalAcc_rate', 'pubRec_diff', 'revolamnt', 'revol_2_income',
       'installment_revol_2_income', 'total_in

In [ ]:
adj_bin = {'installment': [162.0, 251.5, 335.0, 501.0],
           'employmentLength': [0.0, 2.0, 5.0, 10.0],
           'homeOwnership': [1],
           'postCode': [21.0, 272.0],
           'regionCode': [11],
           'title': [4.0, 26.0],
           'n5': [7.0, 11.0],
           'n8': [1.0],
           'issueDate_diff': [2496, 3196],
           'employmentTitle_freq_encode': [4e-06, 3.4e-05, 0.00559, 0.063978],
           'title_freq_encode': [0.004412, 0.4914],
           'total_income': [0.0, 92000.0, 450027.0, 606267.0, 758070.0, 1018000.0],
           'null_num': [4],
}
bins.update(adj_bin)
c2.set_rules(bins)
temp_data = c2.transform(data_)

In [ ]:
# badrate_plot(temp_data, target = 'isDefault', x = 'type', by = 'end_year')
# bin_plot(temp_data, x='end_year', target='isDefault')

#### 4.3 分箱后的特征选择

In [ ]:
selected_data, drop_lst= toad.selection.select(temp_data,target = 'isDefault', empty = 0.5, iv = 0.003,
                                               corr = 0.8, 
                                               return_drop=True, exclude=['type'])

In [ ]:
selected_data.columns

Index(['loanAmnt', 'term', 'subGrade', 'employmentTitle', 'employmentLength',
       'homeOwnership', 'annualIncome', 'verificationStatus', 'purpose', 'dti',
       'ficoRangeLow', 'pubRec', 'revolUtil', 'title', 'n0', 'n2', 'n7', 'n8',
       'n10', 'n13', 'n14', 'issueDate_diff', 'earliesCreditLine_diff',
       'employmentTitle_freq_encode', 'loan_2_income', 'openAcc_totalAcc_rate',
       'revolamnt', 'installment_revol_2_income', 'total_income', 'end_year',
       'isDefault', 'type'],
      dtype='object')

In [ ]:
drop_lst

{'empty': array([], dtype=float64),
 'iv': array(['postCode', 'regionCode', 'delinquency_2years', 'revolBal',
        'totalAcc', 'initialListStatus', 'applicationType', 'policyCode',
        'n4', 'n5', 'n11', 'issueDate_month', 'earliesCreditLine_month',
        'pubRec_diff'], dtype=object),
 'corr': array(['null_num', 'n1', 'n6', 'earliesCreditLine_issueDate_diff', 'n9',
        'loan_2_income_per_year', 'interestRate', 'openAcc',
        'pubRecBankruptcies', 'n12', 'revol_2_income',
        'earliesCreditLine_year', 'title_freq_encode', 'issueDate_year',
        'installment', 'n3', 'installment_2_income', 'ficoRangeHigh',
        'grade'], dtype=object)}

In [ ]:
selected_fea = list(selected_data.columns)
selected_fea.remove('isDefault')
selected_fea.remove('type')
# selected_fea

In [ ]:
# X_train_bined = c2.transform(X_train)
# X_testA_bined = c2.transform(X_testA)

X_train_bined = c2.transform(X_train[selected_fea])
X_testA_bined = c2.transform(X_testA[selected_fea])

#### 4.4 WOE映射

In [ ]:
woe_transer = toad.transform.WOETransformer()
X_train_woe = woe_transer.fit_transform(X_train_bined,y_train.values)
X_testA_woe = woe_transer.transform(X_testA_bined)

# data = pd.concat([dev_slct2_woe,off_woe]

#### 4.5 稳定性筛选特征

In [ ]:
psi_df = toad.metrics.PSI(X_train_woe, X_testA_woe).sort_values(0)

In [ ]:
psi_df[psi_df.values > 0.05]

installment_revol_2_income    0.630002
dtype: float64

In [ ]:
X_train_woe.drop(columns=['installment_revol_2_income'], inplace=True)
X_testA_woe.drop(columns=['installment_revol_2_income'], inplace=True)

#### 4.6 stepwise 选择特征

In [ ]:
X_train_stepwise = toad.selection.stepwise(pd.concat((X_train_woe, y_train), axis=1), target = 'isDefault', direction = 'both', criterion = 'aic')

In [ ]:
X_train_stepwise.drop(columns=['isDefault'], inplace=True)

#### 4.7 输出数据

In [ ]:
X_train = X_train_woe[X_train_stepwise.columns].copy()
X_testA = X_testA_woe[X_train_stepwise.columns].copy()

In [ ]:
X_train.shape, X_testA.shape

((800000, 25), (200000, 25))

## 二、提取、切割数据

In [3]:
# with open('./work/features/X_train.csv', 'rb') as file:
#     X_train = pickle.load(file)

# with open('./work/features/y_train.csv', 'rb') as file:
#     y_train = pickle.load(file)
    
# with open('./work/features/X_testA.csv', 'rb') as file:
#     X_testA = pickle.load(file)

## 三、使用五折交叉验证的CatBoost

In [5]:
from sklearn.model_selection import StratifiedKFold,KFold
import catboost as cb
from catboost import CatBoostClassifier, cv, Pool

In [6]:
data_X = X_train.copy()
data_y = y_train.copy()
data_X_testA = X_testA.copy()

In [7]:
### 标记分类变量
categorical_fea = ['initialListStatus', 'employmentTitle', 'applicationType', 
                   'title', 'policyCode', 'purpose', 'regionCode', 'postCode',
                   'verificationStatus', 'homeOwnership']

In [8]:
for i in data_X.columns:
    if i in categorical_fea:
        data_X[i] = data_X[i].astype('str')

In [9]:
for i in data_X_testA.columns:
    if i in categorical_fea:
        data_X_testA[i] = data_X_testA[i].astype('str')

In [10]:
clfs = []
answers = []
mean_score = 0
cv_scores = []
NFOLD = 5
seed = 2020
CB_INFO_PATH = '/home/aistudio/catboost_info'

In [11]:
kf = StratifiedKFold(n_splits=NFOLD, shuffle=True, random_state=seed)

In [ ]:
from catboost.utils import get_gpu_device_count
print('I see %i GPU devices' % get_gpu_device_count())

I see 0 GPU devices


Error 35 (CUDA driver version is insufficient for CUDA runtime version) ignored while obtaining device count


In [12]:
cv_cat_model = cb.CatBoostClassifier(loss_function='Logloss', eval_metric='AUC', 
                                     iterations=20000, depth=6, learning_rate=0.1,
                                     random_state=2020, od_type="Iter",
                                    #  scale_pos_weight = 4,
#                                      bagging_temperature=0.5, sampling_frequency='PerTree', sampling_unit='Object',
#                                      colsample_bylevel=0.8,
#                                      task_type='GPU', devices=0,
                                     use_best_model=True, metric_period=10)

In [13]:
for fold, (train_index, val_index) in enumerate(kf.split(data_X, data_y)):
    X_train_fold, X_val_fold = data_X.iloc[train_index], data_X.iloc[val_index]
    y_train_fold, y_val_fold = data_y.iloc[train_index], data_y.iloc[val_index]
    
    print("fold:", fold)
    clf = cv_cat_model.fit(X_train_fold, y_train_fold, eval_set=(X_val_fold, y_val_fold),
                           verbose=10, 
                        #    cat_features=categorical_fea
                           )
    clfs.append(clf)

    pred_val_fold = clfs[fold].predict(X_val_fold, prediction_type='Probability',
                                       ntree_end = clfs[fold].get_best_iteration())[:,-1]
    
    print('cat验证的auc:{}'.format(roc_auc_score(y_val_fold, pred_val_fold)))
    mean_score += roc_auc_score(y_val_fold, pred_val_fold) / NFOLD
    cv_scores.append(roc_auc_score(y_val_fold, pred_val_fold))

    pred = clfs[fold].predict(data_X_testA, prediction_type='Probability',
                              ntree_end = clfs[fold].get_best_iteration())[:,-1]
    answers.append(pred)

fold: 0


0:	test: 0.7017044	best: 0.7017044 (0)	total: 242ms	remaining: 1h 20m 35s
10:	test: 0.7125682	best: 0.7125682 (10)	total: 1.85s	remaining: 55m 58s
20:	test: 0.7172409	best: 0.7172409 (20)	total: 3.64s	remaining: 57m 40s
30:	test: 0.7209165	best: 0.7209165 (30)	total: 5.32s	remaining: 57m 5s
40:	test: 0.7228891	best: 0.7228891 (40)	total: 6.85s	remaining: 55m 32s
50:	test: 0.7243252	best: 0.7243252 (50)	total: 8.4s	remaining: 54m 47s
60:	test: 0.7253737	best: 0.7253737 (60)	total: 9.95s	remaining: 54m 11s
70:	test: 0.7262264	best: 0.7262264 (70)	total: 11.5s	remaining: 53m 54s
80:	test: 0.7270756	best: 0.7270756 (80)	total: 13s	remaining: 53m 14s
110:	test: 0.7289797	best: 0.7289797 (110)	total: 17.7s	remaining: 53m
120:	test: 0.7294052	best: 0.7294052 (120)	total: 19.3s	remaining: 52m 54s
130:	test: 0.7299304	best: 0.7299304 (130)	total: 21s	remaining: 53m 4s
140:	test: 0.7304148	best: 0.7304148 (140)	total: 22.7s	remaining: 53m 13s
150:	test: 0.7309164	best: 0.7309164 (150)	total: 24.

0:	test: 0.7015930	best: 0.7015930 (0)	total: 165ms	remaining: 54m 55s
10:	test: 0.7131847	best: 0.7131847 (10)	total: 1.7s	remaining: 51m 34s
20:	test: 0.7181218	best: 0.7181218 (20)	total: 3.2s	remaining: 50m 47s
30:	test: 0.7211225	best: 0.7211225 (30)	total: 4.8s	remaining: 51m 33s
40:	test: 0.7233445	best: 0.7233445 (40)	total: 6.3s	remaining: 51m 6s
50:	test: 0.7248939	best: 0.7248939 (50)	total: 7.89s	remaining: 51m 27s
60:	test: 0.7257785	best: 0.7257785 (60)	total: 9.42s	remaining: 51m 19s
70:	test: 0.7266236	best: 0.7266236 (70)	total: 11s	remaining: 51m 25s
80:	test: 0.7272307	best: 0.7272307 (80)	total: 12.6s	remaining: 51m 29s
90:	test: 0.7277369	best: 0.7277369 (90)	total: 14.1s	remaining: 51m 33s
100:	test: 0.7283545	best: 0.7283545 (100)	total: 15.7s	remaining: 51m 29s
110:	test: 0.7289761	best: 0.7289761 (110)	total: 17.2s	remaining: 51m 28s
120:	test: 0.7295256	best: 0.7295256 (120)	total: 18.8s	remaining: 51m 28s
130:	test: 0.7299656	best: 0.7299656 (130)	total: 20.4

0:	test: 0.7013697	best: 0.7013697 (0)	total: 179ms	remaining: 59m 45s
10:	test: 0.7129690	best: 0.7129690 (10)	total: 1.75s	remaining: 53m 2s
20:	test: 0.7176708	best: 0.7176708 (20)	total: 3.27s	remaining: 51m 53s
30:	test: 0.7209325	best: 0.7209325 (30)	total: 4.79s	remaining: 51m 27s
40:	test: 0.7229372	best: 0.7229372 (40)	total: 6.43s	remaining: 52m 12s
50:	test: 0.7242370	best: 0.7242370 (50)	total: 8.66s	remaining: 56m 27s
60:	test: 0.7252290	best: 0.7252290 (60)	total: 10.1s	remaining: 54m 45s
70:	test: 0.7261511	best: 0.7261511 (70)	total: 11.5s	remaining: 53m 36s
80:	test: 0.7269971	best: 0.7269971 (80)	total: 13.1s	remaining: 53m 42s
90:	test: 0.7276462	best: 0.7276462 (90)	total: 14.6s	remaining: 53m 12s
100:	test: 0.7282541	best: 0.7282541 (100)	total: 16s	remaining: 52m 41s
110:	test: 0.7287033	best: 0.7287033 (110)	total: 17.6s	remaining: 52m 28s
120:	test: 0.7292427	best: 0.7292427 (120)	total: 19s	remaining: 51m 59s
130:	test: 0.7298533	best: 0.7298533 (130)	total: 20

0:	test: 0.7008880	best: 0.7008880 (0)	total: 177ms	remaining: 58m 59s
10:	test: 0.7135799	best: 0.7135799 (10)	total: 1.87s	remaining: 56m 44s
20:	test: 0.7179410	best: 0.7179410 (20)	total: 3.5s	remaining: 55m 30s
30:	test: 0.7210045	best: 0.7210045 (30)	total: 5.16s	remaining: 55m 25s
40:	test: 0.7228672	best: 0.7228672 (40)	total: 6.72s	remaining: 54m 31s
50:	test: 0.7243672	best: 0.7243672 (50)	total: 8.19s	remaining: 53m 24s
60:	test: 0.7253928	best: 0.7253928 (60)	total: 9.65s	remaining: 52m 35s
70:	test: 0.7262158	best: 0.7262158 (70)	total: 11.1s	remaining: 51m 48s
80:	test: 0.7269029	best: 0.7269029 (80)	total: 12.5s	remaining: 51m 20s
90:	test: 0.7275756	best: 0.7275756 (90)	total: 14s	remaining: 51m 3s
100:	test: 0.7280063	best: 0.7280063 (100)	total: 15.4s	remaining: 50m 34s
110:	test: 0.7284676	best: 0.7284676 (110)	total: 16.8s	remaining: 50m 11s
120:	test: 0.7287309	best: 0.7287309 (120)	total: 18.2s	remaining: 49m 54s
130:	test: 0.7292373	best: 0.7292373 (130)	total: 1

0:	test: 0.7016266	best: 0.7016266 (0)	total: 170ms	remaining: 56m 40s
10:	test: 0.7142691	best: 0.7142691 (10)	total: 1.71s	remaining: 51m 48s
20:	test: 0.7186823	best: 0.7186823 (20)	total: 3.17s	remaining: 50m 15s
30:	test: 0.7218542	best: 0.7218542 (30)	total: 4.61s	remaining: 49m 27s
40:	test: 0.7238475	best: 0.7238475 (40)	total: 6.03s	remaining: 48m 55s
50:	test: 0.7254282	best: 0.7254282 (50)	total: 7.53s	remaining: 49m 5s
60:	test: 0.7263322	best: 0.7263322 (60)	total: 9.1s	remaining: 49m 33s
70:	test: 0.7271027	best: 0.7271027 (70)	total: 10.7s	remaining: 50m 4s
80:	test: 0.7277441	best: 0.7277441 (80)	total: 12.9s	remaining: 53m
90:	test: 0.7282766	best: 0.7282766 (90)	total: 14.4s	remaining: 52m 35s
100:	test: 0.7286820	best: 0.7286820 (100)	total: 16s	remaining: 52m 26s
110:	test: 0.7291880	best: 0.7291880 (110)	total: 17.4s	remaining: 52m 4s
120:	test: 0.7296576	best: 0.7296576 (120)	total: 18.9s	remaining: 51m 48s
130:	test: 0.7301907	best: 0.7301907 (130)	total: 20.3s	r

In [14]:
print("cat_scotrainre_list:{}".format(cv_scores))
print("cat_score_mean:{}".format(np.mean(cv_scores)))
print("cat_score_std:{}".format(np.std(cv_scores)))

cat_scotrainre_list:[0.7368389545947767, 0.7374873594288325, 0.7361834779842349, 0.736845861518866, 0.7371542971738236]
cat_score_mean:0.7369019901401067
cat_score_std:0.00043120752861687273


In [15]:
#最终加权平均的预测结果
cat_pre=sum(answers)/NFOLD
cat_pre

array([0.27816621, 0.70941176, 0.86901125, ..., 0.47569488, 0.59665175,
       0.13623801])

In [16]:
submission = pd.DataFrame({'id' : X_testA.index, 'isDefault' : cat_pre})
# submission['isDefault'] = submission['isDefault'].apply(lambda x : x if x > 0 else 0)
# submission['isDefault'] = submission['isDefault'].apply(lambda x : x if x < 1 else 1)
submission.to_csv('./work/cv_catboost/submission_cv_catboost20211219(全训练数据+scale_pos_weight).csv', index = None)

#### 暂时调参优化的效果还没有没有优化的好

## 四、参数调优

In [ ]:
def rm_catboost_info_dir(path):
    # 文件路径
    if os.path.exists(path):
        shutil.rmtree(path)
        # os.rmdir()
        # os.removedirs()
    else:
        print('no such file:%s'%path)  # 则返回文件不存在
    
    pass

In [ ]:
cv_result_global = []

"""定义优化函数"""
def cv_cat_eval(depth, max_leaves, min_data_in_leaf, random_strength, bagging_temperature, 
            #  colsample_bylevel, 
             border_count, l2_leaf_reg, scale_pos_weight):
    # 建立模型
    params = {'loss_function': 'Logloss', 'eval_metric': 'AUC', 'grow_policy': 'Lossguide', #'logging_level': 'Silent',
              'iterations': 3000, 'learning_rate': 0.1,
              'random_state': 2020, 'od_type': "Iter",
              'depth': int(depth), 'max_leaves': int(max_leaves),
              'min_data_in_leaf': int(min_data_in_leaf),
              'random_strength': random_strength,
              'bagging_temperature': round(bagging_temperature, 2),
            #   'colsample_bylevel': min(colsample_bylevel, 0.999), #rsm on GPU is supported for pairwise modes only
              'border_count': int(border_count),
              'l2_leaf_reg': round(l2_leaf_reg, 2),
              'scale_pos_weight': round(scale_pos_weight, 2),
              # 'sampling_frequency': 'PerTree', 'sampling_unit': 'Object',
              'task_type': 'GPU', 'devices': '0', 'pinned_memory_size': '10737418240',
              'thread_count': 2, 'verbose': 500,
              'use_best_model': True, 'metric_period': 10}

    cv_dataset = Pool(data = data_X, label = data_y, cat_features = categorical_fea, thread_count = 2)
    
    rm_catboost_info_dir(CB_INFO_PATH)

    cv_result_cb = cb.cv(cv_dataset,
                         params,
                         fold_count= NFOLD,
                         stratified = True, shuffle=True, partition_random_seed=seed,
                         # plot="True"
                         )

    rm_catboost_info_dir(CB_INFO_PATH)

    global cv_result_global.append(cv_result_cb)

    return cv_result_cb['test-AUC-mean'].max()
    
    # val = cross_val_score(model_cat, data_X, data_y, cv=5, scoring='roc_auc').mean()
    # print(val)
    # return val

In [ ]:
"""定义优化参数"""
cat_opt = BayesianOptimization(cv_cat_eval, {'depth': (4, 8),
                                             'max_leaves': (10, 50),
                                             'min_data_in_leaf': (10,100),
                                             'random_strength': (1e-9, 10),
                                             'bagging_temperature': (0, 10),
                                            #  'colsample_bylevel': (0,1),
                                             'border_count': (1,255),
                                             'l2_leaf_reg': (0, 30),
                                             'scale_pos_weight': (0.1, 5.0)})

"""开始优化"""
cat_opt.maximize(n_iter=20)

|   iter    |  target   | baggin... | border... |   depth   | l2_lea... | max_le... | min_da... | random... | scale_... |
-------------------------------------------------------------------------------------------------------------------------


Custom logger is already specified. Specify more than one logger at same time is not thread safe.Warning: Overfitting detector is active, thus evaluation metric is calculated on every iteration. 'metric_period' is ignored for evaluation metric.


Training on fold [0/5]
0:	learn: 0.6973737	test: 0.6992024	best: 0.6992024 (0)	total: 22ms	remaining: 44s
500:	learn: 0.7410333	test: 0.7402304	best: 0.7402304 (500)	total: 10.3s	remaining: 30.7s
1000:	learn: 0.7458495	test: 0.7418616	best: 0.7418621 (999)	total: 21.4s	remaining: 21.4s
bestTest = 0.7425332367
bestIteration = 1367
Training on fold [1/5]
0:	learn: 0.6987478	test: 0.6968072	best: 0.6968072 (0)	total: 22.4ms	remaining: 44.8s
500:	learn: 0.7423446	test: 0.7352690	best: 0.7352749 (498)	total: 10.4s	remaining: 31.1s
1000:	learn: 0.7471817	test: 0.7366398	best: 0.7366398 (1000)	total: 20.8s	remaining: 20.7s
bestTest = 0.73716712
bestIteration = 1354
Training on fold [2/5]
0:	learn: 0.6978894	test: 0.6990773	best: 0.6990773 (0)	total: 23.6ms	remaining: 47.3s
500:	learn: 0.7415811	test: 0.7378299	best: 0.7378335 (499)	total: 10.9s	remaining: 32.7s
1000:	learn: 0.7465532	test: 0.7392800	best: 0.7392800 (1000)	total: 22.5s	remaining: 22.4s
bestTest = 0.7395449579
bestIteration = 1

0:	learn: 0.6877871	test: 0.6910117	best: 0.6910117 (0)	total: 28.6ms	remaining: 57.1s
500:	learn: 0.7338650	test: 0.7357910	best: 0.7357910 (500)	total: 9.91s	remaining: 29.7s
1000:	learn: 0.7370977	test: 0.7376424	best: 0.7376424 (1000)	total: 19.7s	remaining: 19.6s
1500:	learn: 0.7393346	test: 0.7384690	best: 0.7384690 (1500)	total: 29.9s	remaining: 9.95s
bestTest = 0.7384939194
bestIteration = 1517
Training on fold [1/5]
0:	learn: 0.6901253	test: 0.6874198	best: 0.6874198 (0)	total: 23.6ms	remaining: 47.1s
500:	learn: 0.7349994	test: 0.7312903	best: 0.7312903 (500)	total: 10.1s	remaining: 30.2s
1000:	learn: 0.7383131	test: 0.7328487	best: 0.7328487 (1000)	total: 20s	remaining: 20s
1500:	learn: 0.7405667	test: 0.7337330	best: 0.7337344 (1499)	total: 30.2s	remaining: 10.1s
bestTest = 0.7338304222
bestIteration = 1567
Training on fold [2/5]
0:	learn: 0.6926715	test: 0.6936034	best: 0.6936034 (0)	total: 22.4ms	remaining: 44.7s
500:	learn: 0.7343486	test: 0.7337629	best: 0.7337629 (500)

0:	learn: 0.6967485	test: 0.6981155	best: 0.6981155 (0)	total: 23.3ms	remaining: 46.6s
500:	learn: 0.7449532	test: 0.7412976	best: 0.7413049 (497)	total: 10.3s	remaining: 30.8s
bestTest = 0.7420527935
bestIteration = 782
Training on fold [1/5]
0:	learn: 0.6973364	test: 0.6952449	best: 0.6952449 (0)	total: 22.7ms	remaining: 45.3s
500:	learn: 0.7465829	test: 0.7364337	best: 0.7364337 (500)	total: 11.3s	remaining: 33.9s
bestTest = 0.7371203303
bestIteration = 793
Training on fold [2/5]
0:	learn: 0.6969614	test: 0.6976854	best: 0.6976854 (0)	total: 24.8ms	remaining: 49.6s
500:	learn: 0.7455445	test: 0.7391213	best: 0.7391213 (500)	total: 10.6s	remaining: 31.9s
bestTest = 0.7397499382
bestIteration = 678
Training on fold [3/5]
0:	learn: 0.6977390	test: 0.6969496	best: 0.6969496 (0)	total: 24.5ms	remaining: 49s
500:	learn: 0.7460916	test: 0.7369066	best: 0.7369066 (500)	total: 10.8s	remaining: 32.3s
bestTest = 0.7378203273
bestIteration = 895
Training on fold [4/5]
0:	learn: 0.6971613	test: 

...

0:	learn: 0.6981092	test: 0.6997313	best: 0.6997313 (0)	total: 28.2ms	remaining: 56.5s
500:	learn: 0.7496520	test: 0.7425821	best: 0.7425821 (500)	total: 12.7s	remaining: 38.1s
bestTest = 0.7432335317
bestIteration = 659
Training on fold [1/5]
0:	learn: 0.6984319	test: 0.6960199	best: 0.6960199 (0)	total: 22.9ms	remaining: 45.8s
500:	learn: 0.7512212	test: 0.7370473	best: 0.7370548 (499)	total: 10.3s	remaining: 30.7s
bestTest = 0.7375664711
bestIteration = 726
Training on fold [2/5]
0:	learn: 0.6987209	test: 0.6995873	best: 0.6995873 (0)	total: 23.7ms	remaining: 47.4s
500:	learn: 0.7505484	test: 0.7396188	best: 0.7396188 (500)	total: 10.8s	remaining: 32.2s
bestTest = 0.7403425574
bestIteration = 740
Training on fold [3/5]
0:	learn: 0.6989527	test: 0.6976589	best: 0.6976589 (0)	total: 57.8ms	remaining: 1m 55s
500:	learn: 0.7510740	test: 0.7376627	best: 0.7376694 (498)	total: 11.8s	remaining: 35.2s
bestTest = 0.7385402322
bestIteration = 752
Training on fold [4/5]
0:	learn: 0.6982664	tes

0:	learn: 0.7014653	test: 0.7031414	best: 0.7031414 (0)	total: 25.3ms	remaining: 50.7s
500:	learn: 0.7546649	test: 0.7428470	best: 0.7428597 (498)	total: 13.1s	remaining: 39.2s
bestTest = 0.743103981
bestIteration = 576
Training on fold [1/5]
0:	learn: 0.7028029	test: 0.6999617	best: 0.6999617 (0)	total: 31.5ms	remaining: 1m 2s
500:	learn: 0.7558800	test: 0.7372352	best: 0.7372415 (496)	total: 11.7s	remaining: 34.9s
bestTest = 0.7374916077
bestIteration = 602
Training on fold [2/5]
0:	learn: 0.7017865	test: 0.7016216	best: 0.7016216 (0)	total: 27ms	remaining: 53.9s
500:	learn: 0.7554125	test: 0.7400789	best: 0.7400939 (495)	total: 11.9s	remaining: 35.6s
bestTest = 0.7402095795
bestIteration = 567
Training on fold [3/5]
0:	learn: 0.7021199	test: 0.7005168	best: 0.7005168 (0)	total: 27.9ms	remaining: 55.8s
500:	learn: 0.7556142	test: 0.7375806	best: 0.7375873 (499)	total: 13.3s	remaining: 39.9s
bestTest = 0.7376196682
bestIteration = 528
Training on fold [4/5]
0:	learn: 0.7023120	test: 0

0:	learn: 0.5978414	test: 0.6007463	best: 0.6007463 (0)	total: 23.2ms	remaining: 46.4s
500:	learn: 0.7195112	test: 0.7221501	best: 0.7221501 (500)	total: 10.3s	remaining: 31s
1000:	learn: 0.7258984	test: 0.7277399	best: 0.7277410 (999)	total: 21.5s	remaining: 21.5s
1500:	learn: 0.7297602	test: 0.7307246	best: 0.7307250 (1497)	total: 32.8s	remaining: 10.9s
1999:	learn: 0.7323035	test: 0.7323934	best: 0.7323977 (1990)	total: 43.3s	remaining: 0us
bestTest = 0.7323977053
bestIteration = 1990
Training on fold [1/5]
0:	learn: 0.6226329	test: 0.6202896	best: 0.6202896 (0)	total: 23.6ms	remaining: 47.2s
bestTest = 0.6962838173
bestIteration = 40
Training on fold [2/5]
0:	learn: 0.6243904	test: 0.6271806	best: 0.6271806 (0)	total: 22.6ms	remaining: 45.2s
500:	learn: 0.7189502	test: 0.7192395	best: 0.7192395 (500)	total: 10.4s	remaining: 31.2s
1000:	learn: 0.7266508	test: 0.7260689	best: 0.7260693 (999)	total: 20.7s	remaining: 20.7s
1500:	learn: 0.7302794	test: 0.7288031	best: 0.7288101 (1498)	t

0:	learn: 0.6951346	test: 0.6981334	best: 0.6981334 (0)	total: 35.3ms	remaining: 1m 10s
500:	learn: 0.7414602	test: 0.7383230	best: 0.7383239 (499)	total: 16.2s	remaining: 48.5s
1000:	learn: 0.7487398	test: 0.7398723	best: 0.7398723 (1000)	total: 30.7s	remaining: 30.6s
bestTest = 0.73996526
bestIteration = 1054
Training on fold [1/5]
0:	learn: 0.6970607	test: 0.6947423	best: 0.6947423 (0)	total: 34.9ms	remaining: 1m 9s
500:	learn: 0.7430217	test: 0.7340185	best: 0.7340185 (500)	total: 14.7s	remaining: 43.9s
1000:	learn: 0.7504244	test: 0.7355096	best: 0.7355096 (1000)	total: 30.6s	remaining: 30.5s
bestTest = 0.735632658
bestIteration = 1078
Training on fold [2/5]
0:	learn: 0.6973284	test: 0.6975012	best: 0.6975012 (0)	total: 34.7ms	remaining: 1m 9s
500:	learn: 0.7419611	test: 0.7361212	best: 0.7361266 (498)	total: 14.7s	remaining: 44.1s
1000:	learn: 0.7494245	test: 0.7376235	best: 0.7376251 (999)	total: 29.3s	remaining: 29.3s
bestTest = 0.7377319932
bestIteration = 1099
Training on fol

0:	learn: 0.7044200	test: 0.7067097	best: 0.7067097 (0)	total: 71.3ms	remaining: 2m 22s
500:	learn: 0.7506201	test: 0.7410054	best: 0.7410054 (500)	total: 16s	remaining: 47.8s
bestTest = 0.7418677807
bestIteration = 794
Training on fold [1/5]
0:	learn: 0.7047577	test: 0.7025589	best: 0.7025589 (0)	total: 35.8ms	remaining: 1m 11s
500:	learn: 0.7524938	test: 0.7366547	best: 0.7366653 (496)	total: 14.6s	remaining: 43.6s
bestTest = 0.737562269
bestIteration = 894
Training on fold [2/5]
0:	learn: 0.7045355	test: 0.7048374	best: 0.7048374 (0)	total: 33.2ms	remaining: 1m 6s
500:	learn: 0.7513214	test: 0.7382506	best: 0.7382506 (500)	total: 15.7s	remaining: 47s
bestTest = 0.7389344573
bestIteration = 778
Training on fold [3/5]
0:	learn: 0.7055125	test: 0.7029886	best: 0.7029886 (0)	total: 35.5ms	remaining: 1m 10s
500:	learn: 0.7518526	test: 0.7367084	best: 0.7367084 (500)	total: 14.9s	remaining: 44.6s
bestTest = 0.7373988032
bestIteration = 723
Training on fold [4/5]
0:	learn: 0.7050048	test: 

In [ ]:
scores_global

[]

In [ ]:
"""显示优化结果"""
cat_opt.max

{'target': 0.7398925125598907,
 'params': {'bagging_temperature': 1.220390928659133,
  'border_count': 118.61340421872606,
  'depth': 4.35796527495905,
  'l2_leaf_reg': 22.558363394745143,
  'max_leaves': 33.051299567937996,
  'min_data_in_leaf': 61.869558493987704,
  'random_strength': 9.892362117980387,
  'scale_pos_weight': 1.596558171003779}}

In [ ]:
par_bagging_temperature = 1.220390928659133
par_border_count = 118.61340421872606
par_depth = 4.35796527495905
par_l2_leaf_reg = 22.558363394745143
par_max_leaves = 33.051299567937996
par_min_data_in_leaf = 61.869558493987704
par_random_strength = 9.892362117980387
par_scale_pos_weight = 1.596558171003779

In [ ]:
"""调整一个较小的学习率，并通过cv函数确定当前最优的迭代次数"""

params = {'loss_function': 'Logloss', 'eval_metric': 'AUC', 'grow_policy': 'Lossguide', #'logging_level': 'Silent',
           'iterations': 20000, 'learning_rate': 0.01,
           'random_state': 2020, 'od_type': "Iter",
           'depth': int(par_depth), 'max_leaves': int(par_max_leaves),
           'min_data_in_leaf': int(par_min_data_in_leaf),
           'random_strength': par_random_strength,
           'bagging_temperature': round(par_bagging_temperature, 2),
            #   'colsample_bylevel': min(colsample_bylevel, 0.999), #rsm on GPU is supported for pairwise modes only
            'border_count': int(par_border_count),
            'l2_leaf_reg': round(par_l2_leaf_reg, 2),
            'scale_pos_weight': round(par_scale_pos_weight, 2),
            # 'sampling_frequency': 'PerTree', 'sampling_unit': 'Object',
            'task_type': 'GPU', 'devices': '0', 'pinned_memory_size': '10737418240',
            'thread_count': 2, 'verbose': 500,
            'use_best_model': True, 'metric_period': 10}

cv_dataset = Pool(data = data_X, label = data_y, cat_features = categorical_fea, thread_count = 2)
    
# 文件路径
rm_catboost_info_dir(CB_INFO_PATH)

scores = cb.cv(cv_dataset,
               params,
               fold_count= NFOLD,
               stratified = True, shuffle=True, partition_random_seed=seed,
               # plot="True"
               )

rm_catboost_info_dir(CB_INFO_PATH)

# return scores['test-AUC-mean'].max()

print('迭代次数{}'.format(scores['test-AUC-mean'].argmax() + 1))
print('最终模型的AUC为{}'.format(scores['test-AUC-mean'].max()))

no such file:/home/aistudio/catboost_info
Training on fold [0/5]


0:	learn: 0.6982691	test: 0.6999936	best: 0.6999936 (0)	total: 24.5ms	remaining: 8m 10s
500:	learn: 0.7282381	test: 0.7304803	best: 0.7304803 (500)	total: 10.8s	remaining: 7m
1000:	learn: 0.7335614	test: 0.7353970	best: 0.7353970 (1000)	total: 21.1s	remaining: 6m 41s
1500:	learn: 0.7363580	test: 0.7375832	best: 0.7375832 (1500)	total: 31.4s	remaining: 6m 26s
2000:	learn: 0.7383305	test: 0.7389250	best: 0.7389250 (2000)	total: 42.4s	remaining: 6m 21s
2500:	learn: 0.7398762	test: 0.7398430	best: 0.7398430 (2500)	total: 53.1s	remaining: 6m 11s
3000:	learn: 0.7411936	test: 0.7405412	best: 0.7405412 (3000)	total: 1m 3s	remaining: 5m 58s
3500:	learn: 0.7423085	test: 0.7410316	best: 0.7410316 (3500)	total: 1m 14s	remaining: 5m 51s
4000:	learn: 0.7433035	test: 0.7414340	best: 0.7414340 (4000)	total: 1m 25s	remaining: 5m 40s
4500:	learn: 0.7442085	test: 0.7417240	best: 0.7417240 (4500)	total: 1m 35s	remaining: 5m 29s
5000:	learn: 0.7450779	test: 0.7420126	best: 0.7420126 (5000)	total: 1m 45s	re

In [ ]:
print('迭代次数{}'.format(scores.iterations[scores['test-AUC-mean'].argmax()]))
print('最终模型的AUC为{}'.format(scores['test-AUC-mean'].max()))

迭代次数10173
最终模型的AUC为0.7401699602603913


#### 模型参数已经确定，建立最终模型并对验证集进行验证

In [ ]:
cv_cat_model = cb.CatBoostClassifier(loss_function = 'Logloss', eval_metric = 'AUC', grow_policy = 'Lossguide', #logging_level = 'Silent',
                                     iterations = 10173, learning_rate = 0.01,
                                     random_state = 2020, od_type = "Iter",
                                     depth = int(par_depth), max_leaves = int(par_max_leaves),
                                     min_data_in_leaf = int(par_min_data_in_leaf),
                                     random_strength = par_random_strength,
                                     bagging_temperature = round(par_bagging_temperature, 2),
                                     #   colsample_bylevel = min(colsample_bylevel, 0.999), #rsm on GPU is supported for pairwise modes only
                                     border_count = int(par_border_count),
                                     l2_leaf_reg = round(par_l2_leaf_reg, 2),
                                     scale_pos_weight = round(par_scale_pos_weight, 2),
                                    #  sampling_frequency = 'PerTree', sampling_unit = 'Object',
                                     task_type = 'GPU', devices = '0', pinned_memory_size = '10737418240',
                                     thread_count = 2, verbose = 500,
                                     use_best_model = True, metric_period = 10)

In [ ]:
clfs = []
answers = []
mean_score = 0
cv_scores = []

In [ ]:
for fold, (train_index, val_index) in enumerate(kf.split(data_X, data_y)):
    X_train_fold, X_val_fold = data_X.iloc[train_index], data_X.iloc[val_index]
    y_train_fold, y_val_fold = data_y.iloc[train_index], data_y.iloc[val_index]
    
    print("fold:", fold)
    clf = cv_cat_model.fit(X_train_fold, y_train_fold, eval_set=(X_val_fold, y_val_fold),
                           cat_features=categorical_fea)
    clfs.append(clf)

    pred_val_fold = clfs[fold].predict(X_val_fold, prediction_type='Probability',
                                       ntree_end = clfs[fold].get_best_iteration())[:,-1]
    
    print('cat验证的auc:{}'.format(roc_auc_score(y_val_fold, pred_val_fold)))
    mean_score += roc_auc_score(y_val_fold, pred_val_fold) / NFOLD
    cv_scores.append(roc_auc_score(y_val_fold, pred_val_fold))

    pred = clfs[fold].predict(data_X_testA, prediction_type='Probability',
                              ntree_end = clfs[fold].get_best_iteration())[:,-1]
    answers.append(pred)

fold: 0
0:	learn: 0.6983379	test: 0.6975791	best: 0.6975791 (0)	total: 22.8ms	remaining: 3m 52s
500:	learn: 0.7288806	test: 0.7274155	best: 0.7274155 (500)	total: 11.5s	remaining: 3m 42s
1000:	learn: 0.7342509	test: 0.7320833	best: 0.7320833 (1000)	total: 22.8s	remaining: 3m 28s
1500:	learn: 0.7371736	test: 0.7341881	best: 0.7341881 (1500)	total: 33s	remaining: 3m 10s
2000:	learn: 0.7391452	test: 0.7354156	best: 0.7354156 (2000)	total: 44.1s	remaining: 3m
2500:	learn: 0.7407259	test: 0.7362988	best: 0.7362988 (2500)	total: 55.1s	remaining: 2m 49s
3000:	learn: 0.7420781	test: 0.7369356	best: 0.7369356 (3000)	total: 1m 6s	remaining: 2m 38s
3500:	learn: 0.7432124	test: 0.7374309	best: 0.7374309 (3500)	total: 1m 17s	remaining: 2m 28s
4000:	learn: 0.7442393	test: 0.7378328	best: 0.7378328 (4000)	total: 1m 28s	remaining: 2m 15s
4500:	learn: 0.7451699	test: 0.7381834	best: 0.7381837 (4499)	total: 1m 38s	remaining: 2m 3s
5000:	learn: 0.7460222	test: 0.7384539	best: 0.7384539 (5000)	total: 1m 4

In [ ]:
print("cat_scotrainre_list:{}".format(cv_scores))
print("cat_score_mean:{}".format(np.mean(cv_scores)))
print("cat_score_std:{}".format(np.std(cv_scores)))

cat_scotrainre_list:[0.7400598771690758, 0.740565402180526, 0.739689654884677, 0.7399893015099162, 0.7406614717492315]
cat_score_mean:0.7401931414986853
cat_score_std:0.0003662567765583028


In [ ]:
#最终加权平均的预测结果
cat_pre=sum(answers)/NFOLD
cat_pre

array([0.1189979 , 0.48098931, 0.73869679, ..., 0.27696019, 0.46801406,
       0.04171148])

In [ ]:
submission = pd.DataFrame({'id' : data_X_testA.index, 'isDefault' : cat_pre})
# submission['isDefault'] = submission['isDefault'].apply(lambda x : x if x > 0 else 0)
# submission['isDefault'] = submission['isDefault'].apply(lambda x : x if x < 1 else 1)
submission.to_csv('./work/cv_catboost/submission_cv_catboost20211216(全训练数据、参数优化、五折交叉验证).csv', index = None)

NameError: name 'cat_pre' is not defined

#### 通过5折交叉验证可以发现，模型迭代次数在10173次的时候会停之，那么我们在建立新模型时直接设置最大迭代次数，并使用验证集进行模型预测

In [ ]:
data_X,X_eval,data_y,y_eval=train_test_split(data_X,data_y,test_size=0.2,random_state=1, stratify=data_y)

In [ ]:
final_cat_model = cb.CatBoostClassifier(loss_function = 'Logloss', eval_metric = 'AUC', grow_policy = 'Lossguide', #logging_level = 'Silent',
                                     iterations = 20173, learning_rate = 0.01,
                                     random_state = 2020, od_type = "Iter",
                                     depth = int(par_depth), max_leaves = int(par_max_leaves),
                                     min_data_in_leaf = int(par_min_data_in_leaf),
                                     random_strength = par_random_strength,
                                     bagging_temperature = round(par_bagging_temperature, 2),
                                     #   colsample_bylevel = min(colsample_bylevel, 0.999), #rsm on GPU is supported for pairwise modes only
                                     border_count = int(par_border_count),
                                     l2_leaf_reg = round(par_l2_leaf_reg, 2),
                                     scale_pos_weight = round(par_scale_pos_weight, 2),
                                    #  sampling_frequency = 'PerTree', sampling_unit = 'Object',
                                     task_type = 'GPU', devices = '0', pinned_memory_size = '10737418240',
                                     thread_count = 2, verbose = 500,
                                     use_best_model = True, metric_period = 10)

final_cat_model.fit(data_X, data_y, eval_set=(X_eval, y_eval), cat_features=categorical_fea)

y_pred = final_cat_model.predict(data_X_testA, prediction_type='Probability',
                                 ntree_end = final_cat_model.get_best_iteration())[:,-1]

0:	learn: 0.6986219	test: 0.7093600	best: 0.7093600 (0)	total: 23.8ms	remaining: 7m 59s
500:	learn: 0.7288120	test: 0.7491941	best: 0.7491941 (500)	total: 16.3s	remaining: 10m 38s
1000:	learn: 0.7341437	test: 0.7553470	best: 0.7553470 (1000)	total: 26.8s	remaining: 8m 32s
1500:	learn: 0.7369169	test: 0.7615193	best: 0.7615193 (1500)	total: 37.4s	remaining: 7m 45s
2000:	learn: 0.7388548	test: 0.7669153	best: 0.7669153 (2000)	total: 48.1s	remaining: 7m 16s
2500:	learn: 0.7403077	test: 0.7705658	best: 0.7705658 (2500)	total: 58.7s	remaining: 6m 54s
3000:	learn: 0.7415112	test: 0.7742200	best: 0.7742200 (3000)	total: 1m 10s	remaining: 6m 45s
3500:	learn: 0.7425255	test: 0.7762204	best: 0.7762204 (3500)	total: 1m 22s	remaining: 6m 31s
4000:	learn: 0.7434192	test: 0.7781346	best: 0.7781346 (4000)	total: 1m 34s	remaining: 6m 22s
4500:	learn: 0.7442492	test: 0.7797413	best: 0.7797413 (4500)	total: 1m 45s	remaining: 6m 6s
5000:	learn: 0.7450000	test: 0.7810265	best: 0.7810265 (5000)	total: 1m 5

In [ ]:
submission = pd.DataFrame({'id' : data_X_testA.index, 'isDefault' : y_pred})
# submission['isDefault'] = submission['isDefault'].apply(lambda x : x if x > 0 else 0)
# submission['isDefault'] = submission['isDefault'].apply(lambda x : x if x < 1 else 1)
submission.to_csv('./work/cv_catboost/submission_cv_catboost20211217(全训练数据、参数优化、单独模型).csv', index = None)